In [1]:
!pip install gcloud

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.4/454.4 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gcloud: filename=gcloud-0.18.3-py3-none-any.whl size=602926 sha256=e4c0a1fee2f5fb173521cf52b97fb306ca1caa34c3639e5f9efc029842afeaa8
  Stored in directory: /root/.cache/pip/wheels/7c/30/88/5017af921da3a33af785f0d0fd3e944b845bc62a445a2c2f69
Successfully built gcloud


In [2]:
!gcloud auth application-default login

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=6rlIxjY0gRZjFwoyzK5fckMbP7D3cy&prompt=consent&token_usage=remote&access_type=offline&code_challenge=njEScRcmq9RPzceqzGwlkircNNGSLolt5ygyid41rA0&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AcvDMrA-X5PcME0V31YULrWXlqPdAovRXQpZdebwRqvOI-rBWGb6tyDpB6KI0FGalJElYQ

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Ca

In [3]:
##importando pacotes
import pandas as pd
import numpy as np
import time
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import openpyxl
import csv
import re

In [37]:
#Carrega CSV
df = pd.read_csv('ilostat_genero_latest_period_v3.csv', decimal = ',')
df

,pais,fonte_pesquisa,genero,setor,ano_pesquisa,quantidade
0,Argentina,"LFS - Permanent Household Survey, Urban",Homem,Público,2023,1079515
1,Argentina,"LFS - Permanent Household Survey, Urban",Homem,Privado,2023,4265218
2,Argentina,"LFS - Permanent Household Survey, Urban",Mulher,Público,2023,1332894
3,Argentina,"LFS - Permanent Household Survey, Urban",Mulher,Privado,2023,3192356
4,Bolívia,LFS - Continuous Employment Survey,Homem,Público,2023,270206
5,Bolívia,LFS - Continuous Employment Survey,Homem,Privado,2023,975162
6,Bolívia,LFS - Continuous Employment Survey,Mulher,Público,2023,269827
7,Bolívia,LFS - Continuous Employment Survey,Mulher,Privado,2023,661064
8,Brasil,HS - Continuous National Household Sample Survey,Homem,Público,2023,5142221
9,Brasil,HS - Continuous National Household Sample Survey,Homem,Privado,2023,31427953


In [38]:
#Calcula a quantidade de vínculos totais (público e privado)
df_total = df.groupby(['pais', 'fonte_pesquisa', 'ano_pesquisa'])['quantidade'].sum().reset_index()
df_total

,pais,fonte_pesquisa,ano_pesquisa,quantidade
0,Argentina,"LFS - Permanent Household Survey, Urban",2023,9869983
1,Bolívia,LFS - Continuous Employment Survey,2023,2176259
2,Brasil,HS - Continuous National Household Sample Survey,2023,68277787
3,Chile,LFS - National Employment Survey,2023,6849403
4,Colômbia,LFS - Integrated Household Survey,2023,12164668
5,Estados Unidos,LFS - Current Population Survey,2023,151241930
6,França,LFS - Employment Survey,2023,24881941
7,México,LFS - National Occupation and Employment Survey,2023,40404138
8,Peru,"LFS - Permanent Employment Survey, National",2023,7973973
9,Uruguai,LFS - Continous Household Survey,2023,1168506


In [39]:
#Adiciona as labels faltantes
df_total[['genero', 'setor']] = 'Total'
df_total

,pais,fonte_pesquisa,ano_pesquisa,quantidade,genero,setor
0,Argentina,"LFS - Permanent Household Survey, Urban",2023,9869983,Total,Total
1,Bolívia,LFS - Continuous Employment Survey,2023,2176259,Total,Total
2,Brasil,HS - Continuous National Household Sample Survey,2023,68277787,Total,Total
3,Chile,LFS - National Employment Survey,2023,6849403,Total,Total
4,Colômbia,LFS - Integrated Household Survey,2023,12164668,Total,Total
5,Estados Unidos,LFS - Current Population Survey,2023,151241930,Total,Total
6,França,LFS - Employment Survey,2023,24881941,Total,Total
7,México,LFS - National Occupation and Employment Survey,2023,40404138,Total,Total
8,Peru,"LFS - Permanent Employment Survey, National",2023,7973973,Total,Total
9,Uruguai,LFS - Continous Household Survey,2023,1168506,Total,Total


In [40]:
#Ordena
df_total = df_total[['pais', 'genero', 'setor', 'fonte_pesquisa', 'quantidade', 'ano_pesquisa']]
df_total

,pais,genero,setor,fonte_pesquisa,quantidade,ano_pesquisa
0,Argentina,Total,Total,"LFS - Permanent Household Survey, Urban",9869983,2023
1,Bolívia,Total,Total,LFS - Continuous Employment Survey,2176259,2023
2,Brasil,Total,Total,HS - Continuous National Household Sample Survey,68277787,2023
3,Chile,Total,Total,LFS - National Employment Survey,6849403,2023
4,Colômbia,Total,Total,LFS - Integrated Household Survey,12164668,2023
5,Estados Unidos,Total,Total,LFS - Current Population Survey,151241930,2023
6,França,Total,Total,LFS - Employment Survey,24881941,2023
7,México,Total,Total,LFS - National Occupation and Employment Survey,40404138,2023
8,Peru,Total,Total,"LFS - Permanent Employment Survey, National",7973973,2023
9,Uruguai,Total,Total,LFS - Continous Household Survey,1168506,2023


In [41]:
#Calcular o total público
df_publico = df[df['setor'] == 'Público']
df_publico = df_publico.groupby(['pais', 'fonte_pesquisa', 'ano_pesquisa', 'setor'])['quantidade'].sum().reset_index()
df_publico['genero'] = 'Total'
df_publico = df_publico[['pais', 'genero', 'setor', 'fonte_pesquisa', 'quantidade', 'ano_pesquisa']]
df_publico



,pais,genero,setor,fonte_pesquisa,quantidade,ano_pesquisa
0,Argentina,Total,Público,"LFS - Permanent Household Survey, Urban",2412409,2023
1,Bolívia,Total,Público,LFS - Continuous Employment Survey,540033,2023
2,Brasil,Total,Público,HS - Continuous National Household Sample Survey,12108285,2023
3,Chile,Total,Público,LFS - National Employment Survey,1184521,2023
4,Colômbia,Total,Público,LFS - Integrated Household Survey,914479,2023
5,Estados Unidos,Total,Público,LFS - Current Population Survey,21590847,2023
6,França,Total,Público,LFS - Employment Survey,5751408,2023
7,México,Total,Público,LFS - National Occupation and Employment Survey,6708817,2023
8,Peru,Total,Público,"LFS - Permanent Employment Survey, National",1565816,2023
9,Uruguai,Total,Público,LFS - Continous Household Survey,238840,2023


In [42]:
#merge
df_proporcao = pd.merge(df_publico, df_total, on=['pais', 'genero', 'fonte_pesquisa', 'ano_pesquisa'])
df_proporcao['prop_vinculos_publicos'] = (df_publico['quantidade'] / df_total['quantidade'])
df_proporcao

,pais,genero,setor_x,fonte_pesquisa,quantidade_x,ano_pesquisa,setor_y,quantidade_y,prop_vinculos_publicos
0,Argentina,Total,Público,"LFS - Permanent Household Survey, Urban",2412409,2023,Total,9869983,0.244419
1,Bolívia,Total,Público,LFS - Continuous Employment Survey,540033,2023,Total,2176259,0.248147
2,Brasil,Total,Público,HS - Continuous National Household Sample Survey,12108285,2023,Total,68277787,0.177339
3,Chile,Total,Público,LFS - National Employment Survey,1184521,2023,Total,6849403,0.172938
4,Colômbia,Total,Público,LFS - Integrated Household Survey,914479,2023,Total,12164668,0.075175
5,Estados Unidos,Total,Público,LFS - Current Population Survey,21590847,2023,Total,151241930,0.142757
6,França,Total,Público,LFS - Employment Survey,5751408,2023,Total,24881941,0.231148
7,México,Total,Público,LFS - National Occupation and Employment Survey,6708817,2023,Total,40404138,0.166043
8,Peru,Total,Público,"LFS - Permanent Employment Survey, National",1565816,2023,Total,7973973,0.196366
9,Uruguai,Total,Público,LFS - Continous Household Survey,238840,2023,Total,1168506,0.204398


In [43]:
#formatando para enviar ao BQ
df_proporcao = df_proporcao.drop(['setor_x', 'setor_y', 'quantidade_x', 'quantidade_y', 'genero'], axis=1)
df_proporcao

,pais,fonte_pesquisa,ano_pesquisa,prop_vinculos_publicos
0,Argentina,"LFS - Permanent Household Survey, Urban",2023,0.244419
1,Bolívia,LFS - Continuous Employment Survey,2023,0.248147
2,Brasil,HS - Continuous National Household Sample Survey,2023,0.177339
3,Chile,LFS - National Employment Survey,2023,0.172938
4,Colômbia,LFS - Integrated Household Survey,2023,0.075175
5,Estados Unidos,LFS - Current Population Survey,2023,0.142757
6,França,LFS - Employment Survey,2023,0.231148
7,México,LFS - National Occupation and Employment Survey,2023,0.166043
8,Peru,"LFS - Permanent Employment Survey, National",2023,0.196366
9,Uruguai,LFS - Continous Household Survey,2023,0.204398


In [44]:
df_proporcao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   pais                    11 non-null     object 
 1   fonte_pesquisa          11 non-null     object 
 2   ano_pesquisa            11 non-null     int64  
 3   prop_vinculos_publicos  11 non-null     float64
dtypes: float64(1), int64(1), object(2)
memory usage: 480.0+ bytes


In [45]:
schema=[
 bigquery.SchemaField('pais','STRING',description='Nome do país selecionado'),
 bigquery.SchemaField('fonte_pesquisa','STRING',description='De qual pesquisa nacional foi extraído aquele dado.'),
 bigquery.SchemaField('ano_pesquisa','INTEGER',description='Ano de coleta da informação'),
 bigquery.SchemaField('prop_vinculos_publicos','Float',description='Proporção de vínculos públicos daquele país')
 ]

In [46]:
## Subindo para datalake
client = bigquery.Client(project='repositoriodedadosgpsp')
dataset_ref = client.dataset('perfil_remuneracao')

/usr/local/lib/python3.10/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [47]:
table_ref = dataset_ref.table('ILOSTAT_proporcao_publicos_selecionados_v4') # nome da tabela no padrão FONTE_algo_intuitivo_dado
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df_proporcao, table_ref, job_config=job_config)
job.result()

LoadJob<project=repositoriodedadosgpsp, location=US, id=87c6dfc5-58da-4154-b91f-4ee40055e179>